In [4]:
N_CLONES = 2000
DB_PATH = f"../data/synthetic/raw/cld_{N_CLONES}clones.db"

In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

DB_PATH = "../data/synthetic/raw/cld.db"
FEATURE_PATH = "../data/synthetic/processed/cld_features_with_label.csv"  # already has drop label
OUT_PATH = "../data/synthetic/processed/cld_features_with_labels_3targets.csv"

conn = sqlite3.connect(DB_PATH)
dataset = pd.read_csv(FEATURE_PATH)

dataset.head()

,clone_id,titer_mean,titer_std,titer_min,titer_max,vcd_mean,vcd_std,vcd_min,vcd_max,viability_mean,...,viability_max,aggregation_mean,aggregation_std,aggregation_min,aggregation_max,titer_slope,vcd_slope,viability_slope,aggregation_slope,productivity_drop_pct
0,CLONE_0001,2.538067,0.293036,2.223711,2.964514,1.107619e+07,9.052556e+05,9.779103e+06,1.232258e+07,94.851455,...,97.386709,8.356812,0.403848,7.723455,8.894030,-0.103703,102268.665747,0.169734,-0.019225,0.387063
1,CLONE_0002,0.814721,0.213007,0.537981,1.132518,1.441091e+07,1.047019e+06,1.346070e+07,1.605341e+07,97.551824,...,99.573812,7.337188,0.449925,6.798531,8.011004,-0.005035,279733.381794,0.198861,-0.023819,0.135156
2,CLONE_0003,3.912552,0.208697,3.621956,4.261524,8.684126e+06,5.834120e+05,7.780120e+06,9.384241e+06,94.390688,...,98.774683,2.235352,0.326064,1.745532,2.861836,-0.058117,51853.805966,0.505344,0.038583,0.335258
3,CLONE_0004,0.488369,0.160312,0.212916,0.747609,1.511725e+07,7.817665e+05,1.405453e+07,1.606425e+07,96.380534,...,98.968096,3.860240,0.305128,3.357574,4.286918,-0.026671,278866.456374,0.504642,0.051834,0.590633
4,CLONE_0005,2.238289,0.160672,2.033612,2.459557,1.117195e+07,1.167196e+06,8.874346e+06,1.239030e+07,95.085238,...,96.870619,3.231717,0.405157,2.620058,4.001488,-0.034059,391064.583721,0.233446,-0.129212,0.291757


In [2]:
def fetch_late_labels(conn, clone_list, late_start=26, late_end=30):
    placeholders = ",".join(["?"] * len(clone_list))
    query = f"""
    SELECT p.clone_id,
           AVG(CASE WHEN ar.assay_type='titer' THEN ar.value END) AS late_mean_titer,
           AVG(CASE WHEN ar.assay_type='aggregation' THEN ar.value END) AS late_mean_aggregation
    FROM assay_result ar
    JOIN passage p ON p.passage_id = ar.passage_id
    WHERE p.passage_number BETWEEN ? AND ?
      AND p.clone_id IN ({placeholders})
    GROUP BY p.clone_id
    """
    params = [late_start, late_end] + list(clone_list)
    return pd.read_sql_query(query, conn, params=params)

late_labels = fetch_late_labels(conn, dataset["clone_id"].tolist(), 26, 30)
late_labels.head()

,clone_id,late_mean_titer,late_mean_aggregation
0,CLONE_0001,1.555675,8.137307
1,CLONE_0002,0.704606,7.603613
2,CLONE_0003,2.600837,2.209127
3,CLONE_0004,0.199922,4.120720
4,CLONE_0005,1.585253,3.213549


In [3]:
dataset_3 = dataset.merge(late_labels, on="clone_id", how="left")

print(dataset_3[["productivity_drop_pct","late_mean_titer","late_mean_aggregation"]].isna().mean())
dataset_3.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)

conn.close()

productivity_drop_pct    0.0
late_mean_titer          0.0
late_mean_aggregation    0.0
dtype: float64
Saved: ../data/synthetic/processed/cld_features_with_labels_3targets.csv
